# **Selenium으로 SNU포털 수강편람 크롤링하기**
작성일자 : 2021년 4월 8일

## **❐ Selenium 소개**

셀레니움(Selenium)은 웹드라이버의 API를 통해 브라우저를 제어하기 때문에, 자바스크립트에 의해 동적으로 생성되는 웹페이지의 데이터를 크롤링 할 때 매우 유용하게 사용되는 크롤링 도구이다.

셀레니움을 사용할 때 가장 보편적인 Chrome driver를 사용한다. 아래의 링크에서 다운받을 수 있다.
https://chromedriver.chromium.org/downloads

셀레니움은 웹드라이버의 DOM에서 요소를 찾을 때 다양한 선택자들 중 골라서 사용하면 된다. 아래와 같이 find_element로 시작하는 함수는 조건에 맞는 요소를 하나만 반환한다. 조건을 만족하는 모든 요소를 반환받고 싶은 경우엔 find_elements로 시작하면 된다.

*   find_element()
*   find_element_by_css_selector()
*   find_element_by_xpath()
*   find_element_by_class_name()
*   find_element_by_tag_name()
*   find_element_by_name()
*   find_element_by_id()
*   find_element_by_link_text()
*   find_element_by_partial_link_text()



## **❐ 필요한 라이브러리 불러오기**

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import csv
import getpass

## **❐ 함수 정의하기**

 - 로그인 처리 함수

In [2]:
def fn_loginPage(driver, id_, pw_):

    driver.find_element_by_name('userId').send_keys(id_)
    driver.find_element_by_name('password').send_keys(pw_)
    driver.find_element_by_css_selector('#LoginForm > div > fieldset > a.btn_login').click()

 - 수강편람 검색 함수 (콤보박스 처리)

In [3]:
def fn_search(driver):
    
    Grades = Select(driver.find_element_by_id('srchCptnCorsFg')) #학년 콤보박스
    Grades.select_by_value('C013300002') #석사 선택
    
    Major = Select(driver.find_element_by_id('srchOpenUpDeptCd')) #학과 콤보박스
    #Major.select_by_value('250') #경영대학 선택

    driver.find_element_by_css_selector('#cond00 > a.btn_search_ok').click()
    
    driver.implicitly_wait(time_to_wait=5) #암묵적으로 웹 로딩을 기다려주는 메소드

 - 검색 결과 수집 함수 (페이지 넘기기 처리)

In [4]:
def fn_sugang(driver):

    sugang_len = driver.find_element_by_class_name('fc_o').text #검색건수 가져오기
    pages = (int(sugang_len)//10) + 1 #페이지 수 (한 페이지당 10건씩)
    next_pages = pages//10 #페이지바 넘기는 횟수
    
    #column = ['교과구분', '개설대학', '개설학과', '학년', '교과목번호', '강좌번호', '교과목명(부제명)', '학점-강의-실습', '수업교시', '수업형태', '강의실(동-호)', '주담당교수', '강의계획서', '정원', '수강신청인원']
    
    global sugang_list
    
    if pages <= 10: 
        try:
            sugang_list =[]
            for page in range(1,11):
                driver.find_element_by_xpath('//*[@id="content"]/div/div[3]/div[2]/span/a[%s]'%page).click()      
                time.sleep(2)

                table = driver.find_element_by_class_name('tbl_basic')
                tbody = table.find_element_by_tag_name('tbody')
                tr = tbody.find_elements_by_tag_name('tr')
                
                for row in tr:
                    print(row.text.split("\n"))
        
        except:
            print('done')
        
    else:
        
        i=0
        while i <= next_pages: 
            i += 1
            
            try:
                sugang_list =[]
                for page in range(1,11):
                    driver.find_element_by_xpath('//*[@id="content"]/div/div[3]/div[2]/span/a[%s]'%page).click()      
                    time.sleep(2)

                    table = driver.find_element_by_class_name('tbl_basic')
                    tbody = table.find_element_by_tag_name('tbody')
                    tr = tbody.find_elements_by_tag_name('tr')

                    for row in tr:
                        print(row.text.split("\n"))
                        
                driver.find_element_by_xpath('//*[@id="content"]/div/div[3]/div[2]/a[3]').click()
                time.sleep(2)
            
            except:
                print('done')

## ❐ **브라우저에서 크롤링 실행하기**

In [5]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('headless') #background에서 수행하고자 할 때 사용

driver = webdriver.Chrome(executable_path="/Users/jinsol/Desktop/chromedriver",chrome_options=chrome_options) #크롬드라이버 경로 설정

# 로그인에 필요한 계정정보
id_ = getpass.getpass("아이디를 입력하시오. : ") #getpass를 통해 개인정보를 숨길 수 있음
pw_ = getpass.getpass("패스워드를 입력하시오. : ") 

login_url = "https://my.snu.ac.kr/" #로그인 페이지
driver.get(login_url) # 주소입력하고 enter하는 동작

driver.switch_to_frame('mainFrame')

fn_loginPage(driver, id_, pw_)  # 로그인

driver.find_element_by_xpath('//*[@id="header"]/div[2]/div[1]/ul/li[1]/a').click()

driver.switch_to.window(driver.window_handles[1])
driver.implicitly_wait(time_to_wait=5)

driver.find_element_by_xpath('//*[@id="pgm_container"]/h3[3]').click() #수업/성적 탭
driver.find_element_by_xpath('//*[@id="S0302"]').click() #교과목 탭
driver.find_element_by_xpath('//*[@id="S030201"]').click() #수강편람 탭

time.sleep(5)
driver.switch_to.window(driver.window_handles[2])

fn_search(driver) #검색

fn_sugang(driver) #검색 결과 가져오기

driver.quit()

/Users/jinsol/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


아이디를 입력하시오. : ········
패스워드를 입력하시오. : ········


/Users/jinsol/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use driver.switch_to.frame instead
  del sys.path[0]


['전필 융합과학기술대학원 수리정보과학과 2학년 M0000.006200 001 정보보호법 3-3-0 금(19:00~21:50) 이론 이효원 Y 30 (30) 12 타과생불허']
['전필 자연과학대학 통계학과 1학년 M0000.008700 001 대학원 신입생 세미나 1-1-0 금(10:00~10:50) 이론 025-405 장원철 Y 40 (40) 22 ®통계학과 대학원생만 수강 가능']
['전선 치의학대학원 치의학과 4학년 M0000.010300 001 통일치의학 개론 1-16-0 신터전 Y 15 (15) 16']
['전선 치의학대학원 치의학과 4학년 M0000.017000 001 치과의사의 사명과 인생 1-16-0 명훈 Y 10 (10) 5']
['전선 치의학대학원 치의학과 4학년 M0000.017100 001 성장기환자의 일차교정치료 1-16-0 안석준 Y 10 (10) 6']
['전선 환경대학원 환경조경학과(환경조경학전공) 0학년 M0000.018000 001 광역조경계획 4-4-0 월(14:00~17:50) 이론 082-405 손용훈 Y 12 (12) 13']
['전선 환경대학원 환경조경학과(환경조경학전공) 0학년 M0000.018100 001 생물다양성과 환경계획 2-2-0 월(10:00~11:50) 이론 082-402 송영근 Y 20 (20) 12']
['전선 환경대학원 환경조경학과(환경조경학전공) 0학년 M0000.018400 001 조경식물소재 3-3-0 화(10:00~12:50) 이론 082-402 정우건 Y 12 (12) 6']
['전선 음악대학 음악과(작곡전공) 0학년 M0000.022500 001 해외현장학습 2-1-2 김승근 20 (20) 7']
['전선 생활과학대학 의류학과(의류학전공) 0학년 M0000.029100 001 의류학세미나', '(패션머천다이징) 1-1-0 월(17:00~17:50) 이론 222-225 박주연 Y 50 (50) 14 ⓔ®타전공생수강불허']
['전선 환경대학원 환경계획학과(교통학전공) 0학년 M0000.029

['전선 행정대학원 행정학과 1학년 M2856.001100 001 글로벌시대의 발전정책 3-3-0 목(16:00~18:50) 이론 057-1-421 Jorg Michael Dostal Y 40 (40) 9 ⓔ']
['전선 공학전문대학원 응용공학과(응용공학전공) 1학년 M2866.000400 001 공학자를 위한 물리 3-3-0 금(17:00~19:50) 이론 038-422 윤형식 Y 50 (50) 13 ®수강반 제한, 타 대학(원)생 수강 불허']
['전선 공학전문대학원 응용공학과(응용공학전공) 1학년 M2866.000500 001 엔지니어링 프로젝트 매니지먼트 I 3-3-0 수(10:00~12:50) 이론 038-429 박창우 Y 50 (50) 19']
['전선 공학전문대학원 응용공학과(응용공학전공) 1학년 M2866.000800 001 공학자를 위한 수학 3-3-0 월(14:00~16:50) 이론 038-420 신창수 Y 15 (15) 8']
['전선 공학전문대학원 응용공학과(응용공학전공) 1학년 M2866.000900 001 스마트 산업 시스템 3-3-0 목(10:00~12:50) 이론 038-420 박진우 Y 50 (50) 7 ®수강반 제한, 학부생 수강 불허']
['전선 공학전문대학원 응용공학과(응용공학전공) 1학년 M2866.001200 001 시스템 아키텍쳐 설계 개론 3-3-0 화(14:00~16:50) 이론 038-420 Suh Eun Suk Y 30 (30) 9 ®수강반 제한, 타 대학(원)생 수강 불허']
['전선 공학전문대학원 응용공학과(응용공학전공) 1학년 M2866.001300 001 응용공학특론', '(안전필수제어공학) 3-3-0 금(09:00~11:50) 이론 038-422 김국헌 Y 30 (30) 12 ®수강반 제한, 타 대학(원)생 수강 불허']
['전선 공학전문대학원 응용공학과(응용공학전공) 1학년 M2866.001300 003 응용공학특론', '(화학공학개론) 3-3-0 화(10:00~12:50) 이론 038-420 정광섭 

['논문 국제농업기술대학원 국제농업기술학과 0학년 M2868.006700 002 대학원논문연구 3-3-0 최준원 Y 5 (5) 1 ⓔ']
['논문 국제농업기술대학원 국제농업기술학과 0학년 M2868.006700 003 대학원논문연구 3-3-0 김경훈 5 (5) 3 ⓔ']
['논문 국제농업기술대학원 국제농업기술학과 0학년 M2868.006700 004 대학원논문연구 3-3-0 김도만 Y 5 (5) 5 ⓔ']
['논문 국제농업기술대학원 국제농업기술학과 0학년 M2868.006700 005 대학원논문연구 3-3-0 김주곤 Y 5 (5) 2 ⓔ']
['논문 국제농업기술대학원 국제농업기술학과 0학년 M2868.006700 006 대학원논문연구 3-3-0 박병철 5 (5) 1 ⓔ']
['논문 국제농업기술대학원 국제농업기술학과 0학년 M2868.006700 007 대학원논문연구 3-3-0 허철성 5 (5) 2 ⓔ']
['논문 국제농업기술대학원 국제농업기술학과 0학년 M2868.006700 009 대학원논문연구 3-3-0 강진호 Y 5 (5) 1 ⓔ']
['논문 국제농업기술대학원 국제농업기술학과 0학년 M2868.006700 010 대학원논문연구 3-3-0 박미선 5 (5) 2 ⓔ']
['논문 국제농업기술대학원 국제농업기술학과 0학년 M2868.006700 011 대학원논문연구 3-3-0 박태섭 5 (5) 1 ⓔ']
['논문 국제농업기술대학원 국제농업기술학과 0학년 M2868.006700 013 대학원논문연구 3-3-0 염수청 5 (5) 1 ⓔ']
['논문 국제농업기술대학원 국제농업기술학과 0학년 M2868.006700 014 대학원논문연구 3-3-0 정춘균 Y 5 (5) 2 ⓔ']
['논문 국제농업기술대학원 국제농업기술학과 0학년 M2868.006700 015 대학원논문연구 3-3-0 김갑돈 Y 5 (5) 1 ⓔ']
['논문 국제농업기술대학원 국제농업기술학과 0학년 M2868.006700 016 대학원논문연구 3-3-0 김종근 5 (5) 3 ⓔ']
['논문 국제농업기술대학

['논문 미술대학 디자인학부(디자인전공) 0학년 611.803 007 대학원논문연구 3-3-0 김수정 Y 20 (20) 2']
['논문 미술대학 디자인학부(디자인전공) 0학년 611.803 008 대학원논문연구 3-3-0 김경선 20 (20) 5']
['논문 미술대학 디자인학부(디자인전공) 0학년 611.803 011 대학원논문연구 3-3-0 정의철 Y 20 (20) 2']
['논문 미술대학 디자인학부(디자인전공) 0학년 611.803 013 대학원논문연구 3-3-0 장성연 Y 20 (20) 2']
['논문 미술대학 디자인학부(디자인전공) 0학년 611.803 014 대학원논문연구 3-3-0 이장섭 20 (20) 5']
['논문 미술대학 디자인학부(공예전공) 1학년 611.803 015 대학원논문연구 3-3-0 허보윤 Y 20 (20) 1']
['전선 미술대학 협동과정 미술경영 0학년 614.512 001 전시기획실습 3-3-0 목(10:00~12:50) 이론 083-203 고동연 Y 20 (20) 19']
['전필 음악대학 음악과(작곡전공) 0학년 650.501 001 졸업연주 1-0-2 서정은 Y 80 (80) 12 ®대학원 음악과 석사과정 모든 전공 대상']
['전선 음악대학 음악과 1학년 650.556 001 논문발표 1-0-2 최선아 Y 30 (30) 1']
['전선 음악대학 음악과(작곡·지휘·음악학전공) 0학년 650.561 001 음악분석이론연구', '(조성음악2) 3-3-0 수(14:00~16:50) 이론 220-415 조현리 Y 15 (15) 8']
['전선 음악대학 성악과 1학년 650.562 001 작곡가특강(성)', '(러시아 문헌 연구) 3-3-0 목(13:00~15:50) 이론 053-109 조은혜 20 (20) 8 ®성악과 대학원 전공선택']
['전필 음악대학 성악과 1학년 651.505 001 성악문헌연구', '(후기낭만주의이전문헌) 3-3-0 화(10:00~12:50) 이론 054-211 이윤정 Y 20 (20) 8 ®성악과 석사

['전선 사범대학 협동과정 미술교육전공 1학년 704.540 001 미술교육과 지역사회 3-3-0 화(10:00~12:50) 이론 011-109 김수진 Y 20 (20) 9']
['전선 사범대학 국어교육과 1학년 705.532A 001 국어의미화용교육론연구 3-3-0 월(17:00~19:50) 이론 009-108 남가영 Y 30 (30) 18']
['전선 사범대학 국어교육과 1학년 705.609 001 매체언어교육연습 3-3-0 목(14:00~16:50) 이론 009-109 진가연 Y 30 (30) 19']
['전선 사범대학 국어교육과 1학년 705.611 001 작문교육론연구 3-3-0 금(14:00~16:50) 이론 011-214 김종철 Y 30 (30) 17']
['전선 사범대학 국어교육과 1학년 705.615 001 한국어대조오류분석론연구 3-3-0 월(10:00~12:50) 이론 010-1-501 이성준 Y 30 (30) 38']
['전선 사범대학 국어교육과 1학년 705.620 001 한국문화와 한국어교육연구 3-3-0 월(14:00~16:50) 이론 011-214 고정희 Y 30 (30) 28']
['전선 사범대학 국어교육과 1학년 705.728 001 문학비평교육론연구 3-3-0 수(10:00~12:50) 이론 009-109 진가연 Y 30 (30) 10']
['전선 사범대학 외국어교육과(영어전공) 0학년 707.805 001 연구방법과 영어논문작성 3-3-0 목(16:00~18:50) 이론 Claire Wonnie Jo Y 12 (12) 6 ⓔ외국어교육과 영어전공 외 수강불허']
['전선 사범대학 윤리교육과 2학년 714.523 001 사회와 정의 3-3-0 화(18:30~21:20) 이론 011-111 손경원 Y 30 (30) 13']
['전선 사범대학 윤리교육과 1학년 714.534A 001 현대도덕교육이론연구 3-3-0 수(18:30~21:20) 이론 012-501 정창우 Y 30 (30) 24']
['전선 사범대학 체육교육과 0학년 72

['논문 치의학대학원 치의학과 0학년 861.736 074 논문연구 2-2-0 김태일 10 (10) 1']
['논문 치의학대학원 치의학과 0학년 861.736 075 논문연구 2-2-0 한승현 10 (10) 1']
['논문 치의학대학원 치의학과 0학년 861.736 076 논문연구 2-2-0 KIM RYAN JIN YOUNG(김진영) 10 (10) 1']
['논문 치의학대학원 치의학과 0학년 861.736 077 논문연구 2-2-0 박희경 10 (10) 1']
['논문 치의학대학원 치의학과 0학년 861.736 078 논문연구 2-2-0 김명주 10 (10) 1']
['논문 치의학대학원 치의학과 0학년 861.736 079 논문연구 2-2-0 손원준 10 (10) 1']
['논문 치의학대학원 치의학과 0학년 861.736 081 논문연구 2-2-0 임원희 10 (10) 1']
['논문 치의학대학원 치의학과 0학년 861.736 082 논문연구 2-2-0 권호범 10 (10) 1']
['논문 치의학대학원 치의학과 0학년 861.736 083 논문연구 2-2-0 구기태 10 (10) 1']
['논문 치의학대학원 치의학과 0학년 861.736 084 논문연구 2-2-0 안진수 10 (10) 1']
['논문 치의학대학원 치의학과 0학년 861.736 086 논문연구 2-2-0 한동헌 10 (10) 1']
['논문 치의학대학원 치의학과 0학년 861.736 088 논문연구 2-2-0 서덕규 10 (10) 1']
['논문 치의학대학원 치의학과 0학년 861.736 089 논문연구 2-2-0 박영석 10 (10) 5']
['논문 치의학대학원 치의학과 0학년 861.736 090 논문연구 2-2-0 서광석 10 (10) 1']
['논문 치의학대학원 치의학과 0학년 861.736 091 논문연구 2-2-0 여인성 10 (10) 2']
['논문 치의학대학원 치의학과 0학년 861.736 093 논문연구 2-2-0 윤혜정 10 (10) 1']
['논문 치의학대학원 치의학과 0학년

['전선 행정대학원 공기업정책학과 1학년 925.520 001 공공정책과 갈등관리 3-3-0 월(16:00~18:50) 이론 057-101 권혁주 Y 25 (25) 22 공기업정책학과생만 수강가능']
['전선 환경대학원 환경계획학과 0학년 941.511 001 계획이론 3-3-0 목(14:00~16:50) 이론 082-305 전상인 Y 50 (50) 8']
['전선 환경대학원 환경계획학과(도시및지역계획학전공) 0학년 941.514B 001 도시·지역경제학 3-3-0 월(19:00~21:50) 이론 082-305 이영성 Y 50 (50) 31']
['전선 환경대학원 환경계획학과(도시및지역계획학전공) 0학년 941.521B 001 토지이용계획론 3-3-0 금(10:00~12:50) 이론 082-205 박인권 Y 30 (30) 21']
['전선 환경대학원 환경계획학과(교통학전공) 0학년 941.527 001 교통공학 3-3-0 월(14:00~16:50) 이론 082-304 이영인 Y 20 (20) 12']
['전선 환경대학원 환경계획학과(교통학전공) 0학년 941.531A 001 교통신호운영 3-3-0 월(19:00~21:50) 이론 082-304 이영인 Y 20 (20) 9']
['전선 환경대학원 환경계획학과(환경관리학전공) 0학년 941.551 001 환경학의 최신 이슈 3-3-0 월(14:00~16:50) 이론 082-205 오능환 Y 20 (20) 15']
['전선 환경대학원 환경계획학과(환경관리학전공) 0학년 941.577 001 수질관리 3-3-0 목(10:00~12:50) 이론 220-333 이동수 Y 10 (10) 5']
['전선 환경대학원 환경계획학과(환경관리학전공) 0학년 941.578 001 환경생태학 3-3-0 화(14:00~16:50) 이론 220-333 정수종 Y 20 (20) 14']
['전선 환경대학원 환경계획학과 0학년 941.610 001 지리정보체계 3-2-2 월(14:00~16:50) 이론 220-333 김경민 Y 20 (20) 